In [71]:
PV = PolynomialRing(QQ, ['v%d'%i for i in range(20)])
Ph = PolynomialRing(PV, 'h')
Pd = PolynomialRing(Ph, 'd')
d = Pd.gen()
h = Pd(Ph.gen())
zero = Pd.zero()
V = map(Pd, PV.gens())

def shifted_legendre(n):
    x = var('x')
    return Pd((maxima.legendre_p(n,2*d/h-1).sage()*h**n).expand())

def filter_high_degree(expr, n, depth=2):
    if depth == 0:
        return expr
    s = parent(expr).zero()
    v = d if depth == 2 else h
    for i, coef in enumerate(expr):
        if i > n:
            break
        s += filter_high_degree(coef, n-i, depth-1)*v**i
    return s

In [74]:
def calculateEta(hmax, n):

    numIt = (2*n)//3+1
    dV = sum(shifted_legendre(i)*V[i] for i in range(1, n))

    def calcC(Q, R0):
        C = [[integrate(Pd(Q/2), d)]]
        R0 = Pd(R0)
        q = 0
        while q < numIt:
            if q > 0:
                C.append([zero])
            for m in range(0, hmax//2+1):
                if q == 0:
                    R = R0 if m == 0 else zero
                else:
                    R = filter_high_degree(dV*C[q-1][m], hmax-2*m+3)
                
                C[q].append(
                    integrate(d**m * (
                        R - diff(C[q][m], d, d) - (C[q][m]*V[0] - V[0]*C[q][m])
                    )*Pd(1/2), d)
                       .quo_rem(d**(m+1))[0]
                )
            q += 1
        return C

    Cu = calcC(dV, zero)
    Cv = calcC(zero, dV)

    eta_u = [zero for _ in range(hmax//2+2)]
    eta_up = eta_u[:]
    eta_v = eta_u[:]
    eta_vp = eta_u[:]

    eta_u[0] = Pd.one()
    eta_up[0] = zero
    eta_v[1] = d
    eta_vp[0] = Pd.one()

    for i in range(numIt-1):
        for k in range(hmax//2+1):
            eta_u[k+1] += Cu[i][k] * d**(2*k+1)
            eta_v[k+1] += Cv[i][k] * d**(2*k+1)
        eta_up[0] += Cu[i][0]
        eta_vp[0] += Cv[i][0]
        for k in range(hmax//2):
            eta_up[k+1] += (diff(Cu[i][k], d) + d*Cu[i][k+1]) * d**(2*k+1)
            eta_vp[k+1] += (diff(Cv[i][k], d) + d*Cv[i][k+1]) * d**(2*k+1)

    return [eta_u, eta_v, eta_up, eta_vp]

In [75]:
calculateEta(10, 8)

[[1,
  429/2*v7*d^9 + (-858*v7*h + 66*v6)*d^8 + (1386*v7*h^2 - 231*v6*h + 21*v5)*d^7 + (-1155*v7*h^3 + 315*v6*h^2 - 63*v5*h + 7*v4)*d^6 + (525*v7*h^4 - 210*v6*h^3 + 70*v5*h^2 - 35/2*v4*h + 5/2*v3)*d^5 + (-126*v7*h^5 + 70*v6*h^4 - 35*v5*h^3 + 15*v4*h^2 - 5*v3*h + v2)*d^4 + (14*v7*h^6 - 21/2*v6*h^5 + 15/2*v5*h^4 - 5*v4*h^3 + 3*v3*h^2 - 3/2*v2*h + 1/2*v1)*d^3 + (-1/2*v7*h^7 + 1/2*v6*h^6 - 1/2*v5*h^5 + 1/2*v4*h^4 - 1/2*v3*h^3 + 1/2*v2*h^2 - 1/2*v1*h)*d^2,
  (2178*v6^2 + 9009/2*v5*v7)*d^16 + ((-15246*v6^2 - 63063/2*v5*v7)*h + 1386*v5*v6 + 3003/2*v4*v7)*d^15 + ((94941/2*v6^2 + 98175*v5*v7)*h^2 + (-9009*v5*v6 - 39039/4*v4*v7)*h + 441/2*v5^2 + 462*v4*v6 + 2145/4*v3*v7)*d^14 + ((-86625*v6^2 - 358281/2*v5*v7)*h^3 + (25788*v5*v6 + 55869/2*v4*v7)*h^2 + (-1323*v5^2 - 2772*v4*v6 - 6435/2*v3*v7)*h + 147*v4*v5 + 165*v3*v6 + 429/2*v2*v7)*d^13 + ((205485/2*v6^2 + 849795/4*v5*v7)*h^4 + (-42735*v5*v6 - 92565/2*v4*v7)*h^3 + (6909/2*v5^2 + 14475/2*v4*v6 + 16797/2*v3*v7)*h^2 + (-1617/2*v4*v5 - 1815/2*v3*v6 -

In [76]:
header = "#include \"../matslise.h\"\n\n"
header += "template<typename Scalar>\n"
header += "void calculate_tcoeff_matrix(\n"
header += "        const Scalar &h,\n"
header += "        const Scalar *vs,\n"
header += "        Eigen::Array<Scalar, MATSLISE_ETA_delta, MATSLISE_HMAX_delta> &tDelta,\n"
header += "        Scalar *tH) {\n"

dhmax = 6
dn = 4
hhmax = 8
hn = 6

for i in range(1, 1+max(hn, dn)):
    header += "    const Scalar &v%d = vs[%d];\n"%(i, i)

body = ""
de = calculateEta(dhmax, dn)
print "calculate eta delta"
he = calculateEta(hhmax, hn)
print "calculate eta h"

def expr_to_C(expr):
    print expr
    return ""

for j in range(1+hhmax//2):
    if j < dhmax/2:
        exprs = [filter_high_degree(de[i][j], dhmax+1) for i in range(4)]
        for i in range(dhmax+1):
            body += "    tDelta(%d, %d)"%(j, i)
            body += " << " + ", ".join([expr_to_C(e[i]) for e in exprs]) + ";\n"

calculate eta delta
calculate eta h
1
0
0
1
0
0
-1/2*v3*h^3 + 1/2*v2*h^2 - 1/2*v1*h
0
0
0
3*v3*h^2 - 3/2*v2*h + 1/2*v1
0
0
0
-5*v3*h + v2
0
0
0
5/2*v3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
-1/2*v3*h^3 + 1/2*v2*h^2 - 1/2*v1*h
0
-1/2*v3*h^3 + 1/2*v2*h^2 - 1/2*v1*h
0
3*v3*h^2 - 3/2*v2*h + 1/2*v1
-1/2*v3*h^3 + 1/2*v2*h^2 - 1/2*v1*h
3*v3*h^2 - 3/2*v2*h + 1/2*v1
0
(1/8*v2^2 + 1/4*v1*v3)*h^4 - 1/4*v1*v2*h^3 + 1/8*v1^2*h^2 - 15/2*v3*h + 3/2*v2
3*v3*h^2 - 3/2*v2*h + 1/2*v1
-5*v3*h + v2
0
(-3/4*v2^2 - 7/4*v1*v3)*h^3 + v1*v2*h^2 - 1/4*v1^2*h + 5*v3
-5*v3*h + v2
5/2*v3
0
(13/8*v2^2 + 4*v1*v3)*h^2 - 5/4*v1*v2*h + 1/8*v1^2
5/2*v3
0
0
(-3/2*v2^2 - 15/4*v1*v3)*h + 1/2*v1*v2
0
0
0
0
0
0
0
0
0
0
0
0
0
-3*v3*h^2 + 3/2*v2*h - 1/2*v1
-1/2*v3*h^3 + 1/2*v2*h^2 - 1/2*v1*h
(1/8*v2^2 + 1/4*v1*v3)*h^4 - 1/4*v1*v2*h^3 + 1/8*v1^2*h^2 + 15/2*v3*h - 3/2*v2
3*v3*h^2 - 3/2*v2*h + 1/2*v1
-1/4*v1*v2*h^3 + 1/8*v1^2*h^2 + 15/2*v3*h - 3/2*v2
3*v3*h^2 - 3/2*v2*h + 1/2*v1
(-3/4*v2^2 - 7/4*v1*v3)*h^3 + v1*v2*h^2 - 1/4*v1^2*h - 15/2*v

In [117]:
a = exprs[2]
a.coefficients()

[(-1/2*v3, h^3*d),
 (3*v3, h^2*d^2),
 (-5*v3, h*d^3),
 (5/2*v3, d^4),
 (1/2*v2, h^2*d),
 (-3/2*v2, h*d^2),
 (v2, d^3),
 (-1/2*v1, h*d),
 (1/2*v1, d^2)]

In [14]:
a = shifted_legendre(3)

In [18]:
a.variables()[0]

d

In [30]:
parent(dV//d**2)

Multivariate Polynomial Ring in h, d over Multivariate Polynomial Ring in v0, v1, v2, v3 over Rational Field

In [230]:
a = poly(expr, d, h)

In [232]:
a.

True

In [129]:
a.as_coeff_exponent(h)[1]

0